**Extract Jobs From Monster using Beautiful Soup**

In [ ]:
#Import libraries
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import pdfkit as pdf
import pyodbc
import sqlalchemy
import urllib

In [ ]:
page=req.get("https://www.monster.com/jobs/search?q=&sort=rv&vw=b&re=14&brd=1&cy=AU")
page

In [ ]:
soup=bs(page.content,'html.parser')

In [ ]:
#Looping through the tree to get the data as lists
jobs=[i.get_text() for i in soup.select(".summary .card-header .title")]
companies=[i.get_text() for i in soup.select(".summary .company ")]
locations=[i.get_text() for i in soup.select(".summary .location")]
links=[i['href'] for i in soup.select(".summary a")]

In [ ]:
#converting to dataframes & data cleanup
mnstr_au_jobs=pd.DataFrame({"Job":jobs,"Company":companies,"Location":locations,"Links":links})
mnstr_au_jobs.replace(['\n','\r'],'',regex=True,inplace=True)
mnstr_au_jobs.head(20)

**Exporting extracted data to various formats**

In [ ]:
#01_to CSV format
mnstr_au_jobs.to_csv(r"C:\Users\Sabeesh\Desktop\WFH_Python\01_Training\06102020\mnstr_au_jobs.csv")

In [ ]:
#02 To Excel 
mnstr_au_jobs.to_excel(r"C:\Users\Sabeesh\Desktop\WFH_Python\01_Training\06102020\mnstr_au_jobs.xlsx")

In [ ]:
#03_To_HTML
mnstr_au_jobs.to_html(r"C:\Users\Sabeesh\Desktop\WFH_Python\01_Training\06102020\mnstr_au_jobs.html")

In [ ]:
#04_To_MySql_Database
#params=pyodbc.connect('Driver={SQL Server};SERVER=LAPTOP-PI6VNN85\SQLEXPRESS;DATABASE=UK_HR_Jobs;Trusted_connection=True')
#cursor = conn.cursor()
params = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=LAPTOP-PI6VNN85\SQLEXPRESS;DATABASE=UK_HR_Jobs;Trusted_connection=True")
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

mnstr_au_jobs.to_sql('mnstr_au_jobs', con=engine, if_exists='replace', index=False)

In [ ]:
#05_To_PDF
html_file=r"C:\Users\Sabeesh\Desktop\WFH_Python\01_Training\06102020\mnstr_au_jobs.html"
pdf_file=r"C:\Users\Sabeesh\Desktop\WFH_Python\01_Training\06102020\mnstr_au_jobs1.pdf"
path_wkhtmltopdf = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe'
config = pdf.configuration(wkhtmltopdf=path_wkhtmltopdf)
pdf.from_file(html_file,pdf_file,configuration=config)